# 스타벅스 매장 지점명, 주소, 번호 정보 수집

In [6]:
from selenium import webdriver as wb #셀레니움 웹드라이버 임포트
from bs4 import BeautifulSoup as bs  #뷰티풀숲 임포트
import time #타임
import pandas as pd #판다스

In [4]:
url = 'http://www.istarbucks.co.kr/store/store_map.do' #주소설정
#driver = wb.Chrome('c:/informs/chromedriver.exe') #크롬 드라이버 설정
driver = wb.Chrome() #크롬 드라이버 설정
driver.get(url) #드라이버를 통해 웹 정보 수신

In [5]:
#지역검색 버튼 클릭




btn_search = driver.find_element_by_class_name('loca_search')
#btn_search = driver.find_element_by_xpath('//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[2]/h3/a')
btn_search.click()

In [6]:
#광주지역 클릭






li = driver.find_element_by_css_selector('ul.sido_arae_box > li + li + li')
li.click()

In [7]:
#광주지역에서 전체지역 클릭






allArea = driver.find_element_by_css_selector('ul.gugun_arae_box > li')
allArea.click()

In [9]:
soup = bs(driver.page_source,'html.parser')#BeautifulSoup으로 웹 페이지 해석

In [10]:
#지점명, 주소, 번호 수집







name_list = []
addr_list = []
tel_list = []

# names = soup.select('li.quickResultLstCon > strong')
names = soup.select('#mCSB_3_container > .quickSearchResultBoxSidoGugun > li > strong')
# addrs = soup.findAll('p',class_='result_details')
addrs = soup.select('#mCSB_3_container > .quickSearchResultBoxSidoGugun > li > strong + p')
print(len(names), len(addrs))

for index in range(len(names)):
    name_list.append(names[index].text)
    addr_list.append(addrs[index].text[:-12])
    tel_list.append(addrs[index].text[-12:])


57 57


In [11]:
dic = {'name':name_list, 'address':addr_list, 'tel':tel_list}
df = pd.DataFrame(dic)
df.head()

,name,address,tel
0,광주운남DT,광주광역시 광산구 임방울대로 156 (운,남동)1522-3232
1,광주선운DT,광주광역시 광산구 선운로 20번길,63 1522-3232
2,광주산정DT,광주광역시 광산구 용아로 342 (산,정동)1522-3232
3,광주흑석DT,광주광역시 광산구 사암로 367 (산,정동)1522-3232
4,광주송정역,광주광역시 광산구 상무대로205번길 6 (송,정동)1522-3232


## 스타벅스 모든 음료 정보 가져오기

In [36]:
url = 'http://www.istarbucks.co.kr/menu/index.do'

driver = wb.Chrome()
driver.get(url)

In [37]:
#자세히 보기 클릭







driver.find_element_by_class_name('menu_drink_btn01').click()

In [38]:
#영양정보 보기 클릭







driver.find_element_by_class_name('a2').click()

In [39]:
#BeautifulSoup으로 웹 페이지 해석






soup = bs(driver.page_source,'html.parser')

In [40]:
#항목타이틀, 모든 음료정보 수집







drink_cols = soup.select('table.coffeeInfo.mb60 th',limit=7)
drink_infos = soup.select('table.coffeeInfo.mb60 td')

In [ ]:
"""방법1"""
df_col = []
for index in range(1,len(drink_cols)):
    df_col.append(drink_cols[index].text)
    print(drink_cols[index].text)

In [24]:
"""방법2 - 과정 복잡도 높으므로 패스"""
# 태그를 제외하고 값만 넣음
drink_info_texts = [] 
for tag in drink_infos:
    drink_info_texts.append(tag.text)
    
# 데이터프레임으로 만들기위해 2차원으로 만들기
drink_2dim_list = []
for i in range(len(drink_info_texts)//7):
    drink_2dim_list.append(drink_info_texts[i*7:i*7+7])
    
# 컬럼 추출
df_col = [] 
for index in range(len(drink_cols)):
    df_col.append(drink_cols[index].text)
    
#컬럼만 있는 빈 데이터프레임
df = pd.DataFrame(drink_2dim_list, columns=df_col)
df.head(1)

#컬럼 메뉴를 인덱스로
df.set_index("메뉴", inplace=True)
df.head(1)

,칼로리(Kcal),당류(g),단백질(g),나트륨(mg),포화지방(g),카페인(mg)
메뉴,,,,,,
나이트로 바닐라 크림,75,10,1,20,2,245


칼로리(Kcal)
당류(g)
단백질(g)
나트륨(mg)
포화지방(g)
카페인(mg)


In [42]:
#컬럼만 있는 빈 데이터프레임
df = pd.DataFrame(columns=df_col)
df

,칼로리(Kcal),당류(g),단백질(g),나트륨(mg),포화지방(g),카페인(mg)


In [43]:
#한줄씩 추가 연습함
df.loc[drink_infos[0].text] = [drink_infos[1].text,drink_infos[2].text,drink_infos[3].text,drink_infos[4].text,drink_infos[5].text,drink_infos[6].text]
df

,칼로리(Kcal),당류(g),단백질(g),나트륨(mg),포화지방(g),카페인(mg)
나이트로 바닐라 크림,75,10,1,20,2,245


In [44]:
#반복문 돌려서 모든 행 넣어줌
for index in range(int(len(drink_infos)/7)):
    df.loc[drink_infos[(index*7)].text] = [drink_infos[(index*7)+1].text,drink_infos[(index*7)+2].text,drink_infos[(index*7)+3].text,drink_infos[(index*7)+4].text,drink_infos[(index*7)+5].text,drink_infos[(index*7)+6].text]

In [45]:
df.head()

,칼로리(Kcal),당류(g),단백질(g),나트륨(mg),포화지방(g),카페인(mg)
나이트로 바닐라 크림,75,10,1,20,2,245
나이트로 쇼콜라 클라우드,55,7,1,20,2,245
나이트로 콜드 브루,5,0,0,5,0,245
돌체 콜드 브루,265,29,8,115,9,150
바닐라 크림 콜드 브루,125,11,3,58,6,150


In [46]:
df.to_csv('drink.csv')

# 웹 페이지 스크롤링
### -->  Keys.PAGE_DOWN 활용

In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [52]:
url = 'https://www.google.com/search?source=hp&ei=-QZ1W6XKH4ui8AWe6oOgAQ&q=빅데이터&gs_l=psy-ab.3..0l10.3446.6828.0.6945.11.9.0.1.1.0.178.797.1j5.6.0....0...1c.1j4.64.psy-ab..6.5.589...0i131k1j0i10k1.0.onTVEC6H7No'
driver = wb.Chrome()
driver.get(url)

In [57]:
#웹 페이지를 3번 스크롤링하시오.









#<body>태그 검색
body = driver.find_element_by_tag_name('body')
for i in range(3):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)

# 유튜브 정보 수집
### --> 영상제목 / 영상주소 / 조회수 

In [8]:
url_main = 'https://www.youtube.com'
url_sub = '/results?search_query='
query = '무한도전'
driver = wb.Chrome()
driver.get(url_main+url_sub+query)

In [9]:
#현재 페이지에서 60번 스크롤링
for index in range(5):
    body = driver.find_element_by_tag_name('body')
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

In [10]:
soup = bs(driver.page_source,'html.parser')

In [19]:
#영상제목 / 영상주소 / 조회수 /게시일/영상길이/작성자

nums = []
titles = []
video_urls = []
views = []
days=[]


titles = soup.findAll('a',id='video-title')
views = soup.select('#metadata-line > span')
#days=select('div:nth-child(2)')

#데이터 확인 샘플 1개
print('영상 개수 :',len(titles))
print('영상 제목 :',titles[0].text.strip())
print('영상 주소 :',url_main+titles[0]['href'])
#print('조회수 : ',views[0].text)
view_list = []
for i in range(len(views)):
    #print(views[i].text) #조회수, 게시일 모두 포함
    #조회수와 게시일 분리
    if i%2==0: #조회수만 저장
        view_list.append(views[i].text)
        #view_list.append(views[i].text[4:])
        #print(views[i].text[4:])
        
        
title_list = []
url_list = []
#day_list=[]
for index in range(len(titles)):
    title_list.append(titles[index].text.strip())
    url_list.append(url_main+titles[index]['href'])
  #  day_list.append(days[index].text.strip())
len(title_list), len(url_list), len(view_list)#, len(day_list)


영상 개수 : 38
영상 제목 : [8月의 무도] 👉나영이의 선택👈 끝나지 않는 😱지옥에서 온 매력 발산🕺 “개그동아리 MT” 2편 infinite challenge
영상 주소 : https://www.youtube.com/watch?v=Bfr6W8NcMhA


(38, 38, 38)

In [53]:
#view_list.append("가라로 넣어버리기~")

In [18]:
#데이터 프레인으로 확인
data = {'제목':title_list,'주소':url_list,'조회수':view_list}
youtubes = pd.DataFrame(data)
youtubes.head()

,제목,주소,조회수
0,[8月의 무도] 👉나영이의 선택👈 끝나지 않는 😱지옥에서 온 매력 발산🕺 “개그동아...,https://www.youtube.com/watch?v=Bfr6W8NcMhA,조회수 6.3만회
1,괜찮아요 안 물어요~ 쓸개도 항문도 일부 없지만 광기는 있는 노홍철 모음.zip |...,https://www.youtube.com/watch?v=Ped3crmspe8,조회수 12만회
2,"[8月의 무도] 우리 재석이가 화가 (많이) 났어요👿 흑화한 재석, 그리고 배신과 ...",https://www.youtube.com/watch?v=XyPJq4ukAyE,조회수 34만회
3,🎉경 존버당 축🎉 정준하 결혼 프로젝트? 사실은 멤버들의 씹뜯맛즐 티키타카! ＂짱가...,https://www.youtube.com/watch?v=AjzzKfVg260,조회수 41만회
4,[8月의 무도] 🌟나영이를 지켜라!🌟 79학번 전설의 선배들과 함께 떠나는 “개그동...,https://www.youtube.com/watch?v=lvq2Lv2ixeo,조회수 19만회


In [ ]:
""" key 제어
    NULL = '\ue000'
    CANCEL = '\ue001'
    HELP = '\ue002'
    BACKSPACE = '\ue003'
    BACK_SPACE = BACKSPACE
    TAB = '\ue004'
    CLEAR = '\ue005'
    RETURN = '\ue006'
    ENTER = '\ue007'
    SHIFT = '\ue008'
    LEFT_SHIFT = SHIFT
    CONTROL = '\ue009'
    LEFT_CONTROL = CONTROL
    ALT = '\ue00a'
    LEFT_ALT = ALT
    PAUSE = '\ue00b'
    ESCAPE = '\ue00c'
    SPACE = '\ue00d'
    PAGE_UP = '\ue00e'
    PAGE_DOWN = '\ue00f'
    END = '\ue010'
    HOME = '\ue011'
    LEFT = '\ue012'
    ARROW_LEFT = LEFT
    UP = '\ue013'
    ARROW_UP = UP
    RIGHT = '\ue014'
    ARROW_RIGHT = RIGHT
    DOWN = '\ue015'
    ARROW_DOWN = DOWN
    INSERT = '\ue016'
    DELETE = '\ue017'
    SEMICOLON = '\ue018'
    EQUALS = '\ue019'

    NUMPAD0 = '\ue01a'
    NUMPAD1 = '\ue01b'
    NUMPAD2 = '\ue01c'
    NUMPAD3 = '\ue01d'
    NUMPAD4 = '\ue01e'
    NUMPAD5 = '\ue01f'
    NUMPAD6 = '\ue020'
    NUMPAD7 = '\ue021'
    NUMPAD8 = '\ue022'
    NUMPAD9 = '\ue023'
    MULTIPLY = '\ue024'
    ADD = '\ue025'
    SEPARATOR = '\ue026'
    SUBTRACT = '\ue027'
    DECIMAL = '\ue028'
    DIVIDE = '\ue029'

    F1 = '\ue031'
    F2 = '\ue032'
    F3 = '\ue033'
    F4 = '\ue034'
    F5 = '\ue035'
    F6 = '\ue036'
    F7 = '\ue037'
    F8 = '\ue038'
    F9 = '\ue039'
    F10 = '\ue03a'
    F11 = '\ue03b'
    F12 = '\ue03c'

    META = '\ue03d'
    COMMAND = '\ue03d'
"""